# Biblioteki Pythona w analizie danych
### Tomasz Rodak

Lab 9

---



## Sieci konwolucyjne - podstawy

### 1.

Oblicz $\mathbf{I} \ast \mathbf{K}$ dla:

\begin{equation*}
\mathbf{I} = \begin{bmatrix}
2 & 5 & -3 & 0 \\
0 & 6 & 0 & -4 \\
-1 & -3 & 0 & 2 \\
5 & 0 & 0 & 3
\end{bmatrix}
\text{ oraz } \mathbf{K} = \begin{bmatrix}
-2 & 0 \\
4 & 6
\end{bmatrix}
\end{equation*}

### 2.

Rozważmy fragment sieci konwolucyjnej o architekturze:

**Wejście:** Obraz $x$ o rozmiarze $1 \times 224 \times 224$

| Nr | Warstwa            | Parametry                                                                      |
| -- | ------------------ | ------------------------------------------------------------------------------ |
| 1  | $\mathrm{Conv}$ C1 | filtr $5\times5$, stride=1, padding=0 i padding=2 (warianty), 6 filtrów, ReLU  |
| 2  | $\mathrm{Conv}$ C2 | filtr $3\times3$, stride=1, padding=0 i padding=1 (warianty), 16 filtrów, ReLU |



Oblicz:
1. Rozmiar wyjścia z warstwy C1.
2. Rozmiar wyjścia z warstwy C2.
3. Rozmiar okna (*receptive field*) na obrazie $x$ "widzianego" przez filtr konwolucyjny z warstwy C2.
4. Dokładną liczbę parametrów (zarówno wag, jak i biasów) w warstwach C1 oraz C2.

Zbuduj ten fragment sieci konwolucyjnej w PyTorch i zweryfikuj poprawność obliczeń.

Rozwiąż to samo zadanie dla architektury:

**Wejście:** Obraz $x$ o rozmiarze $1 \times 224 \times 224$
| Nr | Warstwa            | Parametry                                                                      |
| -- | ------------------ | ------------------------------------------------------------------------------ |
| 1  | $\mathrm{Conv}$ C1 | filtr $5\times5$, stride=2, padding=0 i padding=2 (warianty), 6 filtrów, ReLU  |
| 2  | $\mathrm{Conv}$ C2 | filtr $3\times3$, stride=2, padding=0 i padding=1 (warianty), 16 filtrów, ReLU |

In [2]:

import torch
import torch.nn as nn
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1, padding=0)
    self.relu = nn.ReLU()
  def forward(self, x):
    x = self.relu(self.conv1(x))
    x = self.relu(self.conv2(x))
    return x

In [3]:
x = torch.randn(1, 1, 224, 224)
model = CNN()
output = model(x)
print(output.shape)

torch.Size([1, 16, 218, 218])


In [12]:
for name, p in model.named_parameters():
  print(f"{name}: {p.shape}")

conv1.weight: torch.Size([6, 1, 5, 5])
Parameter containing:
tensor([[[[-0.1565,  0.0200,  0.1507,  0.0156,  0.1141],
          [ 0.0379, -0.1168, -0.0216, -0.0654, -0.1577],
          [ 0.0203, -0.0672, -0.1193, -0.1741,  0.0855],
          [-0.0814, -0.1067,  0.1888,  0.1712,  0.1651],
          [ 0.0945,  0.1821,  0.1812,  0.0195, -0.1835]]],


        [[[-0.1233, -0.1901,  0.1713,  0.1442, -0.0121],
          [-0.1856, -0.1404, -0.0516,  0.1030, -0.1854],
          [-0.0515,  0.0212,  0.1999, -0.1181, -0.1029],
          [ 0.1748,  0.0684,  0.1163,  0.0312, -0.0030],
          [ 0.0826, -0.1501, -0.0227,  0.1934,  0.0991]]],


        [[[ 0.0975,  0.0914,  0.0842, -0.0973, -0.0145],
          [ 0.0337, -0.1224, -0.1702, -0.1398,  0.1504],
          [-0.0817, -0.1151, -0.0380, -0.0188, -0.1448],
          [-0.1042,  0.1874,  0.1330,  0.1010, -0.0180],
          [ 0.1714, -0.0797,  0.0944, -0.0167,  0.1018]]],


        [[[ 0.1332, -0.1211, -0.0174,  0.1633,  0.0625],
          [-0.1

In [13]:
pip install torchinfo

In [14]:
import torchinfo
torchinfo.summary(model, input_size=(1, 1, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [1, 16, 218, 218]         --
├─Conv2d: 1-1                            [1, 6, 220, 220]          156
├─ReLU: 1-2                              [1, 6, 220, 220]          --
├─Conv2d: 1-3                            [1, 16, 218, 218]         880
├─ReLU: 1-4                              [1, 16, 218, 218]         --
Total params: 1,036
Trainable params: 1,036
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 49.37
Input size (MB): 0.20
Forward/backward pass size (MB): 8.41
Params size (MB): 0.00
Estimated Total Size (MB): 8.61

### 3.


Dany jest fragment sieci konwolucyjnej o architekturze:

**Wejście:** Obraz $x$ o rozmiarze $1 \times 224 \times 224$.

| Nr | Warstwa            | Parametry                                                                      |
| -- | ------------------ | ------------------------------------------------------------------------------ |
| 1  | $\mathrm{Conv}$ C1 | filtr $5\times5$, stride=1, padding=0, 6 filtrów, ReLU                        |
| 2  | MaxPool P1         | okno $2\times2$, stride=2                                                      |
| 3  | $\mathrm{Conv}$ C2 | filtr $3\times3$, stride=1, padding=0, 16 filtrów, ReLU                       |
| 4  | MaxPool P2         | okno $2\times2$, stride=2                                                      |



Oblicz:
1. Rozmiary wyjścia z każdej kolejnej warstwy.
2. Rozmiar okna na obrazie $x$ "widzianego" przez okno poolingu z warstwy P1.
3. Rozmiar okna na obrazie $x$ "widzianego" przez filtr konwolucyjny z warstwy C2.
4. Rozmiar okna na obrazie $x$ "widzianego" przez okno pooling z warstwy P2.
5. Dokładną liczbę parametrów w sieci.

Zbuduj ten fragment sieci konwolucyjnej w PyTorch i zweryfikuj poprawność obliczeń.

### 4.

Zaproponuj własną architekturę sieci CNN do klasyfikacji obrazów CIFAR-10. Są to obrazy kolorowe o rozmiarze $32 \times 32$ pikseli i 10 klasach.

Wykonaj testowy trening, aby upewnić się, że dane są poprawnie wczytywane i architektura działa. Dane można wczytać z repozytoriów PyTorch:

```python
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
```

### 5.

Oblicz ręcznie liczbę parametrów w sieci VGG16. Architekturę sieci znajdziesz w [C. Bishop, Deep Learning](https://www.bishopbook.com/), strona 300.

Potwierdź poprawność obliczeń w PyTorch. Model z losowymi wagami (szybciej się ładuje) możesz pobrać przy pomocy:

```python
import torch
import torchvision.models as models

vgg16 = models.vgg16(pretrained=False)
```

Zobacz [torchvision.models](https://pytorch.org/vision/stable/models.html#torchvision.models.vgg16).

### 6.

Zaimplementuj w PyTorch własną sekwencyjną sieć konwolucyjną (bez pretrenowanych wag) do klasyfikacji zbioru Fashion-MNIST. Architektura powinna zawierać:

1. Co najmniej 2 warstwy konwolucyjne
2. Warstwę `BatchNorm2d` po każdej konwolucji
3. Funkcje aktywacji `ReLU`
4. Pooling do redukcji wymiarowości
5. Klasyfikator w pełni połączony na końcu sieci

Zadanie obejmuje:
- Implementację modelu
- Przygotowanie danych treningowych i testowych
- Napisanie funkcji trenującej
- Ewaluację dokładności na zbiorze testowym